# test note


* jupyterはコンテナ起動すること
* テストベッド一式起動済みであること


In [1]:
!pip install --upgrade pip
!pip install --force-reinstall ../lib/ait_sdk-0.1.3-py3-none-any.whl

Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (20.2.3)
Processing /workdir/root/lib/ait_sdk-0.1.3-py3-none-any.whl
  Using cached nbformat-5.0.7-py3-none-any.whl (170 kB)
  Using cached numpy-1.19.2-cp36-cp36m-manylinux2010_x86_64.whl (14.5 MB)
  Using cached Keras-2.4.3-py2.py3-none-any.whl (36 kB)
  Using cached nbconvert-6.0.7-py3-none-any.whl (552 kB)
Processing /root/.cache/pip/wheels/4c/1f/18/c066f071b2c2b638e40889c5595bc6b61fbe45eb475d29d61d/psutil-5.7.2-cp36-cp36m-linux_x86_64.whl
Processing /root/.cache/pip/wheels/46/6d/cc/73a126dc2e09fe56fcec0a7386d255762611fbed1c86d3bbcc/py_cpuinfo-7.0.0-py3-none-any.whl
  Using cached ipython_genutils-0.2.0-py2.py3-none-any.whl (26 kB)
  Using cached jsonschema-3.2.0-py2.py3-none-any.whl (56 kB)
  Using cached traitlets-4.3.3-py2.py3-none-any.whl (75 kB)
  Using cached jupyter_core-4.6.3-py2.py3-none-any.whl (83 kB)
Processing /root/.cache/pip/wheels/e5/9d/ad/2ee53cf262cba1ffd8afe1487eef788ea3f260b7e6232a80fc

In [2]:
from pathlib import Path
import pprint
from ait_sdk.test.hepler import Helper
import json

In [3]:
# settings cell

# mounted dir
root_dir = Path('/workdir/root/ait')

ait_name='alyz_regression_dist_1var_treemap_tf2.3'
ait_version='0.1'

ait_full_name=f'{ait_name}_{ait_version}'
ait_dir = root_dir / ait_full_name

td_name=f'{ait_name}_test'

# (dockerホスト側の)インベントリ登録用アセット格納ルートフォルダ
current_dir = %pwd
with open(f'{current_dir}/config.json', encoding='utf-8') as f:
    json_ = json.load(f)
    root_dir = json_['host_ait_root_dir']
    is_container = json_['is_container']
invenotory_root_dir = f'{root_dir}\\ait\\{ait_full_name}\\local_qai\\inventory'

# entry point address
# コンテナ起動かどうかでポート番号が変わるため、切り替える
if is_container:
    backend_entry_point = 'http://host.docker.internal:8888/qai-testbed/api/0.0.1'
    ip_entry_point = 'http://host.docker.internal:8888/qai-ip/api/0.0.1'
else:
    backend_entry_point = 'http://host.docker.internal:5000/qai-testbed/api/0.0.1'
    ip_entry_point = 'http://host.docker.internal:6000/qai-ip/api/0.0.1'

# aitのデプロイフラグ
# 一度実施すれば、それ以降は実施しなくてＯＫ
is_init_ait = True

# インベントリの登録フラグ
# 一度実施すれば、それ以降は実施しなくてＯＫ
is_init_inventory = True


In [4]:
helper = Helper(backend_entry_point=backend_entry_point, 
                ip_entry_point=ip_entry_point,
                ait_dir=ait_dir,
                ait_full_name=ait_full_name)

In [5]:
from glob import glob
path = Path(str(ait_dir) + '/deploy/container')
print(path)
print(Path(path / 'ait.manifest.json').exists())
files = glob(str(path.joinpath(f'**/ait.manifest.json')), recursive=True)
print(files)
if len(files) == 0:
    print(f'not found {file_name} in zip.')
elif len(files) > 1:
    print(f'{file_name} must be one exists in zip.')
print(files[0])
helper._find_file(path, 'ait.manifest.json')

/workdir/root/ait/alyz_regression_dist_1var_treemap_tf2.3_0.1/deploy/container
False
['/workdir/root/ait/alyz_regression_dist_1var_treemap_tf2.3_0.1/deploy/container/repository/ait.manifest.json']
/workdir/root/ait/alyz_regression_dist_1var_treemap_tf2.3_0.1/deploy/container/repository/ait.manifest.json


'/workdir/root/ait/alyz_regression_dist_1var_treemap_tf2.3_0.1/deploy/container/repository/ait.manifest.json'

In [6]:
# health check

helper.get_bk('/health-check')
helper.get_ip('/health-check')

<Response [200]>
{'Code': 0, 'Message': 'alive.'}
<Response [200]>
{'Code': 0, 'Message': 'alive.'}


<Response [200]>

In [7]:
# create ml-component
res = helper.post_ml_component(name=f'MLComponent_{ait_full_name}', description=f'Description of {ait_full_name}', problem_domain=f'ProbremDomain of {ait_full_name}')
helper.set_ml_component_id(res['MLComponentId'])

<Response [200]>
{'MLComponentId': 3,
 'Result': {'Code': 'P22000', 'Message': 'add ml-component success.'}}


In [8]:
# deploy AIT
if is_init_ait:
    helper.deploy_ait_non_build()
else:
    print('skip deploy AIT')

<Response [200]>
{'Code': 'M00001', 'Message': 'Add AIT manifest success'}
<Response [200]>
{'Code': 'D00001', 'Message': 'Deploy success'}


In [9]:
res = helper.get_data_types()
model_data_type_id = [d for d in res['DataTypes'] if d['Name'] == 'model'][0]['Id']
dataset_data_type_id = [d for d in res['DataTypes'] if d['Name'] == 'dataset'][0]['Id']
res = helper.get_file_systems()
unix_file_system_id = [f for f in res['FileSystems'] if f['Name'] == 'UNIX_FILE_SYSTEM'][0]['Id']
windows_file_system_id = [f for f in res['FileSystems'] if f['Name'] == 'WINDOWS_FILE'][0]['Id']

In [10]:
# add inventories

if is_init_inventory:
    inv1_name = helper.post_inventory('dataset_for_verification', dataset_data_type_id, windows_file_system_id, 
                                      f'{invenotory_root_dir}\\dataset_for_verification\\dataset_for_verification.csv',
                                      'dataset for verification', ['csv'])
    inv2_name = helper.post_inventory('categories', dataset_data_type_id, windows_file_system_id, 
                                      f'{invenotory_root_dir}\\categories\\categories.csv',
                                      'category variables of dataset', ['csv'])
else:
    print('skip add inventories')

<Response [200]>
{'result': {'Code': 'I22000', 'Message': 'append Inventory success.'}}
<Response [200]>
{'result': {'Code': 'I22000', 'Message': 'append Inventory success.'}}


In [11]:
# get ait_json and inventory_jsons

res_json = helper.get_bk('/QualityMeasurements/RelationalOperators', is_print_json=False).json()
eq_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '=='][0])
nq_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '!='][0])
gt_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '>'][0])
ge_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '>='][0])
lt_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '<'][0])
le_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '<='][0])

res_json = helper.get_bk('/testRunners', is_print_json=False).json()
ait_json = [j for j in res_json['TestRunners'] if j['Name'] == ait_name][-1]

inv_1_json = helper.get_inventory(inv1_name)
inv_2_json = helper.get_inventory(inv2_name)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [12]:
# add teast_descriptions

helper.post_td(td_name, 2,
               quality_measurements=[],
               target_inventories=[
                {"Id":1, "InventoryId": inv_1_json['Id'], "TemplateInventoryId": ait_json['TargetInventories'][0]['Id']},
                {"Id":2, "InventoryId": inv_2_json['Id'], "TemplateInventoryId": ait_json['TargetInventories'][1]['Id']}
               ],
               test_runner={
                "Id":ait_json['Id'],
                "Params":[]
               })

<Response [200]>
{'Result': {'Code': 'T22000', 'Message': 'append test description success.'}}


In [13]:
# get test_description_jsons
td_1_json = helper.get_td(td_name)

<Response [200]>


In [14]:
# run test_descriptions
helper.post_run_and_wait(td_1_json['Id'])

<Response [200]>
{'Job': {'Id': '2', 'StartDateTime': '2020-10-07 14:46:07.800272+09:00'},
 'Result': {'Code': 'R12000', 'Message': 'job launch success.'}}
[{'Id': 5,
  'Result': 'OK',
  'ResultDetail': 'No Measure',
  'Status': 'DONE',
  'TestDescriptionID': 6}]


In [15]:
res_json = helper.get_td_detail(td_1_json['Id'])
pprint.pprint(res_json)

<Response [200]>
{'Result': {'Code': 'T32000', 'Message': 'get detail success.'},
 'TestDescriptionDetail': {'Id': 6,
                           'Name': 'alyz_regression_dist_1var_treemap_tf2.3_test',
                           'Opinion': '',
                           'QualityDimension': {'Id': 2, 'Name': 'データ設計の十分性'},
                           'QualityMeasurements': [],
                           'TargetInventories': [{'DataType': {'Id': 1,
                                                               'Name': 'dataset'},
                                                  'Description': 'dataset for '
                                                                 'verification',
                                                  'Id': 8,
                                                  'Name': 'alyz_regression_dist_1var_treemap_tf2.3_0.1_dataset_for_verification',
                                                  'TemplateInventoryId': 7},
                                          

In [16]:
# generate report
res = helper.post_report(td_1_json['Id'])
pprint.pprint(res)

<Response [200]>
{'OutParams': {'ReportUrl': 'http://127.0.0.1:8888/qai-testbed/api/0.0.1/download/126'},
 'Result': {'Code': 'D12000', 'Message': 'command invoke success.'}}
{'OutParams': {'ReportUrl': 'http://127.0.0.1:8888/qai-testbed/api/0.0.1/download/126'},
 'Result': {'Code': 'D12000', 'Message': 'command invoke success.'}}
